In [1]:
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ['HF_HOME'] = '<PATH_TO>'

rocket_dir = os.path.split(os.path.abspath(os.curdir))[:-1]
rocket_src_dir = os.path.join(*rocket_dir, 'src')
rocket_dir = os.path.join(*rocket_dir)

print(f'Appending to sys.path:', [rocket_dir, rocket_src_dir])

sys.path.append(rocket_dir)
sys.path.append(rocket_src_dir)

import torch
import torch.nn as nn
import numpy as np
import torchvision
import copy
import gymnasium

from PIL import Image
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    GPTQConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

from torch.distributions.categorical import Categorical
from accelerate import Accelerator

Appending to sys.path: ['/data01/gbonetta/rocket', '/data01/gbonetta/rocket/src']


/home/gbonetta/miniconda3/envs/crafter_smart/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.5.2 (SDL 2.28.2, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.distributions.categorical import Categorical

In [3]:
LOCAL_NUM_ENVS = 1
SEED = 1234
MODEL='giobin/Qwen-VL-Chat-Int4-fromImageList' #"Qwen/Qwen-VL-Chat-Int4"
torch.cuda.manual_seed = SEED

In [4]:
is_crafter = False 
from src.environments.minigrid_env import make_minigrid_env
runs_directory = 'runs_directory_prova'
envs = gymnasium.vector.SyncVectorEnv(
            [make_minigrid_env(
                runs_directory,
                'MiniGrid-DoorKey-8x8-v0', #'MiniGrid-Fetch-8x8-N3-v0',#'MiniGrid-DoorKey-5x5-v0', #'MiniGrid-LockedRoom-v0','MiniGrid-LavaGapS7-v0', 'MiniGrid-PutNear-8x8-N3-v0', #'MiniGrid-PutNear-8x8-N3-v0','MiniGrid-BlockedUnlockPickup-v0', 'MiniGrid-LavaCrossingS11N5-v0', 'MiniGrid-DoorKey-8x8-v0' 'MiniGrid-SimpleCrossingS11N5-v0' 'MiniGrid-LavaCrossingS9N3-v0' 'MiniGrid-SimpleCrossingS11N5-v0'
                possible_actions_list=['forward','pickup','toggle','opt_left','opt_right','opt_back'],  #left right forward pickup toggle
                fov=1,
                fixed_orientation=False,
                no_step_description=False,
                seed=1 + i *100,
                save_video=False,
                save_video_every=100,
                save_stats=False
            )
            for i in range(1)
            ])

action_enum = envs.envs[0].action_enum
print(list(action_enum))

[<action_enum.forward: 'move forward'>, <action_enum.pickup: 'pick up'>, <action_enum.toggle: 'toggle'>, <action_enum.opt_left: 'move left'>, <action_enum.opt_right: 'move right'>, <action_enum.opt_back: 'move back'>]


/home/gbonetta/miniconda3/envs/crafter_smart/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FrozenLakeText-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/gbonetta/miniconda3/envs/crafter_smart/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment Hanoi3Disk-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/gbonetta/miniconda3/envs/crafter_smart/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment Hanoi4Disk-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/gbonetta/miniconda3/envs/crafter_smart/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.width to get variables from other wrappers is deprecated and will be removed in v1.0, t

In [5]:
from src.agents.QWen_agent import QWenAgent
agent = QWenAgent(MODEL,
            action_enum=action_enum, 
            is_lora=True,
            padding_side="right", 
            num_prompt_images=1, 
            use_text_description=True, 
            gradient_ckpt=False)
agent.network.to('cuda')
agent.critic.to('cuda')

action_to_string = [a.value for a in action_enum]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/home/gbonetta/miniconda3/envs/crafter_smart/lib/python3.10/site-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.
/home/gbonetta/miniconda3/envs/crafter_smart/lib/python3.10/site-packages/transf

trainable params: 112,197,632 || all params: 3,292,734,208 || trainable%: 3.407430570235689


In [ ]:
def test_inference(value_prompt_template, action_template, temperature, steps, print_logs=100):
    
    tot_reward = 0

    next_obs, infos = envs.reset(seed=SEED)
    next_obs = torch.Tensor(next_obs).to('cuda')
    next_text_obs = infos.get('obs', [None]*LOCAL_NUM_ENVS)

    for step in range(steps):
        with torch.no_grad():
            res = agent.get_action_and_value(next_obs, 
                                                    value_prompt_template=value_prompt_template,
                                                    action_template=action_template,
                                                    text_description=next_text_obs, 
                                                    temperature=temperature, 
                                                    generate_actions=True,
                                                    normalization_by_words=False,
                                                    use_full_seq_logits=True,
                                                    advanced_action_matching=False)
            action, logprob, value = res['action'], res['log_prob'], res['values']

            to_pil = transforms.ToPILImage()
            pil_image = to_pil(transforms.functional.invert(next_obs[0].permute(2, 0, 1)))

            best_action = action.cpu().numpy()
            next_obs, reward, terminations, truncations, infos = envs.step(best_action)
            next_text_obs = infos.get('obs', [None]*LOCAL_NUM_ENVS)    
            next_obs = torch.Tensor(next_obs).to('cuda')

            tot_reward += reward

            if step % print_logs == 0:
                print(f"\n\n####################################### STEP {step} #######################################")
                print(f"this is the env image:\n")
                display(pil_image)
                print(f"-- the prompt describing the image is:\n")
                print(f"{res['just_context_prompts']=}\n")
                print(f"-- the prompt augmented with the first action description is:\n")
                print(f"{res['prompts'][0]=}\n")
                print(f"-- given this situation the agent \'randomly\' chose:\n")
                print(f"action number: {best_action} -> {action_to_string[best_action.item()]}")
                print(f"-- the actual action logits were:\n")
                print(f"{res['action_logits']}\n") 
                print(f"{res['log_prob']}\n") 
                print(f"-- performing the action {best_action} in the env, resulted this:\n")
                print(f"reward: {reward}, terminations: {terminations}, truncations: {truncations}\n")
                print(f"total reward: {tot_reward}\n")
    return tot_reward


In [ ]:
tot_steps = 100
TEMPERATURE = 'max_logit' #'max_logit'
#value_prompt_template = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPicture 1: <img>/placeholder_frame.png</img>\nYou are an agent in a minigrid game.\nRules:\nyou can pick up the key only if it is 1 step ahead of you {}\nWhat's the next best action? Possible actions are:\n- go forward\n- pick up\n- toggle\n- move left\n- move right\n- move back\n\nGame rules:\n- you can pick up the key only if it is 1 step ahead of you.\n- the toggle action may serve to open the door.<|im_end|>\n<|im_start|>assistant\n"
#value_prompt_template = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPicture 1: <img>/placeholder_frame.png</img>\nYou are an agent in a minigrid game.\n {}\nWhat's the next best action? Possible actions are:\n- move forward\n- pick up\n- toggle\n- move left\n- move right\n- move back\n\nGame rules:\n- you can pick up the key only if it is 1 step ahead of you.\n- the toggle action may serve to open the door.<|im_end|>\n<|im_start|>assistant\n"
value_prompt_template = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPicture 1: <img>/placeholder_frame.png</img>\nYou are the red triangle in this minigrid game.\n {}\nGame rules:\n- you can pick up the key only if it is 1 step ahead of you.\n- the toggle action may serve to open the door.\nWhat's the next best action?<|im_end|>\n<|im_start|>assistant\n"
action_template= "Based on the given information, the next best action would be to {}"

tot_reward = test_inference(value_prompt_template=value_prompt_template,
            action_template=action_template,
            temperature=TEMPERATURE,
            steps=tot_steps,
            print_logs=100)

print(f"TOTAL REWARD -> {tot_reward=}")

In [ ]:
import itertools

v_s = [
    "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPicture 1: <img>/placeholder_frame.png</img>\nI am the red triangle in this minigrid game in the picture.\n {}\nWhat's the next best action?<|im_end|>\n<|im_start|>assistant\n",
    "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPicture 1: <img>/placeholder_frame.png</img>\nI am the red triangle in this minigrid game in the picture.\n {}\nPossible game actions are:\n- move forward\n- pick up\n- toggle\n- move left\n- move right\n- move back\nWhat's the next best action?<|im_end|>\n<|im_start|>assistant\n",
    "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPicture 1: <img>/placeholder_frame.png</img>\nI am the red triangle in this minigrid game in the picture.\n {}\nPossible game actions are:\n- move forward\n- pick up\n- toggle\n- move left\n- move right\n- move back\n\nGame rules:\n- I can pick up the key only if it is 1 step ahead of me.\n- the toggle action may serve to open the door.\nWhat's the next best action?<|im_end|>\n<|im_start|>assistant\n",
    "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPicture 1: <img>/placeholder_frame.png</img>\nI am the red triangle in this minigrid game in the picture.\n#### BEGIN GAME STATE DESCRIPTION ####\n {}\n#### END GAME STATE DESCRIPTION ####\nPossible actions are:\n- move forward\n- pick up\n- toggle\n- move left\n- move right\n- move back\n\nGame rules:\n- I can pick up the key only if it is 1 step ahead of me in the GAME STATE DESCRIPTION\n- the toggle action may serve to open the door.\nWhat's the next best action?<|im_end|>\n<|im_start|>assistant\n",
    "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPicture 1: <img>/placeholder_frame.png</img>\nI am the red triangle in this minigrid game in the picture.\n#### BEGIN GAME STATE DESCRIPTION ####\n {}\n#### END GAME STATE DESCRIPTION ####\nPossible actions are:\n- move 1 step forward\n- pick up the object\n- toggle the object\n- move 1 step left\n- move 1 step right\n- move 1 step back\n\nGame rules:\n- I can pick up the key only if it is 1 step ahead of me in the game state description\n- the toggle action may serve to open the door.\n- if the key is behind me in the game state description, I should move back.\nWhat's the next best action?<|im_end|>\n<|im_start|>assistant\n"
]
a_s = [
    " {}",
    "Based on the given information, the next best action would be to {}",
    "Based on the image and the given information, the next best action is to {}"
]

#all combinations
template_combinations = list(itertools.product(v_s, a_s))
print(template_combinations)
print(len(template_combinations))

In [ ]:
agent.use_text_description = True

In [ ]:
TEMPERATURE = 'max_logit'
TOT_STEPS = 1000

for ix, (v,a) in enumerate(template_combinations):
    tot_reward = test_inference(value_prompt_template=v,
                action_template=a,
                temperature=TEMPERATURE,
                steps=TOT_STEPS,
                print_logs=100)
    print(f"{TOT_STEPS=}, {TEMPERATURE=}\n{v=}\n{a=}\n{tot_reward=}\n\n")

In [ ]:
#Questa cella può essere usata per far generare il modello. Può servire per vedere come creare l'action_template per esempio. o per capire che cosa Qwen capisce dell'immagine.
#L'immagine usata è l'ultima che deriva dalla cella sopra dove è stato fatto girare nell'env

t = ["<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nPicture 1: <img>/placeholder_frame.png</img>\nYou are the red agent in this minigrid game on the image.\n\nYour mission is:\n\nuse the key to open the door and then get to the goal.\nWhat is the next best move?<|im_end|>\n<|im_start|>assistant\n",
     "this is anither prompt",
     "this is anither prompt",
     "this is anither asfgdaf asivovnaoa"]

inputs = agent.tokenizer(t, return_tensors='pt',tokenize=False, padding=True).to('cuda')
print(inputs)
context_prompt_lens = torch.sum(inputs.attention_mask, dim=1)
context_split = context_prompt_lens.tensor_split(2)
print(len(context_split)) #[1, 288]
print(context_split)
forward_out = agent.network(**inputs, image_list=None)
#print(forward_out)
#pred = agent.network.generate(**inputs, image_list=next_obs.permute(0, 3, 1, 2))
response = agent.tokenizer.decode(pred.cpu()[0], skip_special_tokens=True)
print(response)
#response = tokenizer.decode(pred.cpu()[1], skip_special_tokens=True)
#print(response)